In [ ]:
%matplotlib inline

In [ ]:
import os
import pickle
from math import floor
from random import shuffle
from statistics import variance
import math

from itertools import chain
from itertools import product
from itertools import groupby
from operator import itemgetter

import pandas as pd
import numpy as np

import fiona
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from mpl_toolkits.basemap import Basemap
from shapely.geometry import Point, Polygon, MultiPoint
from descartes import PolygonPatch

import matplotlib.colors as mpl_colors
from random import randint
import time

from geopy.distance import vincenty

from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import Birch
from sklearn.metrics import silhouette_score

# Creating environment

In [ ]:
img_width_inches = 7.22 * 3
img_height_inches = 5.25 * 3

In [ ]:
with open('./data_routes_pickle/cell_id_lac_info', 'rb') as f:
    station_dict = pickle.load(f)

### Reading map (really long operation)

In [ ]:
m.readshapefile(
    'data_maps_input/spb',
    'city_polygons',
    drawbounds=False)

df_map = pd.DataFrame({
    'poly': [Polygon(polygon_coords) for polygon_coords in m.city_polygons]})

In [ ]:
df_map['patches'] = df_map['poly'].map(lambda x: PolygonPatch(
    x,
    fc='black',
    ec='grey', lw=0.55, alpha=0.6,
    zorder=2))

### Creating basemap, setting up scale etc.

In [ ]:
shp = fiona.open('data_maps_input/spb.shp')
bds = shp.bounds
shp.close()

ll = bds[0], bds[1]
ur = bds[2], bds[3]
coords = list(chain(ll, ur))
w, h = coords[2] - coords[0], coords[3] - coords[1]
zoom_out_frac = -0.3

m = Basemap(
    projection='tmerc',
    lon_0=30.5,
    lat_0=60.,
    ellps='WGS84',
    llcrnrlon=coords[0] + (coords[2] - coords[0]) * 0.06 - zoom_out_frac * w,
    llcrnrlat=coords[1] - zoom_out_frac * h,
    urcrnrlon=coords[2] + (coords[2] - coords[0]) * 0.06 + zoom_out_frac * w,
    urcrnrlat=coords[3] + zoom_out_frac * h,
    lat_ts=0,
    resolution='i',
    suppress_ticks=True)

In [ ]:
map_points = pd.Series(
    [Point(m(mapped_x, mapped_y)) for mapped_x, mapped_y in 
     [(station_dict[i][0], station_dict[i][1]) for i in station_dict]])

station_points = MultiPoint(list(map_points.values))

# Clustering experiements

In [ ]:
with open('data_routes_pickle/routes_coord_cur', 'rb') as f:
    routes_coord_cur = pickle.load(f)

with open('data_routes_pickle/sim_matrix_sim_segments_1_mod23_cumul', 'rb') as f:
    sim_matrix = np.array(pickle.load(f))

In [ ]:
print(len(sim_matrix))
print(len(routes_coord_cur))
# print(cluster_count_cur)

In [ ]:
sim_matrix

In [ ]:
print(np.unravel_index(np.argmax(sim_matrix), sim_matrix.shape))
print(np.amax(sim_matrix))

### Top most similar routes via sim_matrix

In [ ]:
max_points_count = 10
max_idxs_raveled = (-sim_matrix).argsort(axis=None)[:max_points_count]
max_idxs_unravled = np.unravel_index(max_idxs_raveled, sim_matrix.shape)
max_sim_route_idxs = np.array(max_idxs_unravled).T
max_sim_route_idxs

### Distance matrix from similarity one

In [ ]:
dist_matrix = [[0 for x in range(len(sim_matrix))] for y in range(len(sim_matrix))] 
for i in range(len(sim_matrix)):
    for j in range(len(sim_matrix[0])):
        if i != j:
            dist_matrix[i][j] = 135 - sim_matrix[i][j]

### Flattening routes (e.g. for point-based clustering methods)

In [ ]:
routes_coord_cur_flat = [[num for coords in route for num in coords] for route in routes_coord_cur]

### Clustering

In [ ]:
cluster_count_cur = 19

with open('data_routes_pickle/dist_matrix', 'wb') as f:
    pickle.dump(dist_matrix, f)

with open('data_routes_pickle/cluster_count_cur', 'wb') as f:
    pickle.dump(cluster_count_cur, f)

with open('data_routes_pickle/idxs', 'wb') as f:
    pickle.dump(idxs, f)

In [ ]:
# algo = KMeans(n_clusters=cluster_count, random_state=42)
# algo = AgglomerativeClustering(n_clusters=cluster_count_cur)
# algo = DBSCAN(eps=110, min_samples=2, metric='precomputed')
algo = Birch(threshold=124, branching_factor=6, n_clusters=21)

idxs = algo.fit_predict(dist_matrix)

for idx in idxs:
    print(idx, end=' ')

# plt.hist(idxs, range(int(len(idxs) / 2)))
plt.hist(idxs, range(0, cluster_count + 1))
plt.show()

route_idxs_per_cluster = [[] for i in range(21)]
for r_i, c_i in enumerate(idxs):
    route_idxs_per_cluster[c_i].append(r_i)

print(max([len(i) for i in route_idxs_per_cluster]))

# for route_idx, cluster_idx in enumerate(idxs):
#     if cluster_idx == 1:
#         print(route_idx, end=' ')

In [ ]:
var_min = math.inf
state_var_min = -1

for cluster_count in range(9, 10):
#     kmeans = KMeans(n_clusters=cluster_count, random_state=42)
    kmeans = AgglomerativeClustering(n_clusters=cluster_count)
    idxs = kmeans.fit_predict(dist_matrix)
    
    var_cur = variance(idxs)
    print(cluster_count, var_cur)
    if var_min > var_cur:
        var_min = var_cur
    
    plt.hist(idxs, range(0, cluster_count + 1))
    plt.show()

# var_min

In [ ]:
# BIRCH params bf

var_min = math.inf

min_max_clust_count = 500
min_max_clust_count_br_factor = -1
min_max_clust_count_threshold = -1
min_max_clust_count_cluster_count = -1
for cluster_count in range(15, 16):
    for br_factor in range(6, 300):
        for threshold in range(3, 130):
        #     algo = KMeans(n_clusters=cluster_count, random_state=42)
        #     algo = AgglomerativeClustering(n_clusters=cluster_count)
            algo = Birch(threshold=threshold, branching_factor=br_factor, n_clusters=cluster_count)
            idxs = algo.fit_predict(dist_matrix)

            route_idxs_per_cluster = [[] for i in range(cluster_count)]
            for r_i, c_i in enumerate(idxs):
                route_idxs_per_cluster[c_i].append(r_i)

            max_clust_count_cur = max([len(i) for i in route_idxs_per_cluster])

            if max_clust_count_cur < 70:
                if max_clust_count_cur < min_max_clust_count:
                    min_max_clust_count = max_clust_count_cur
                min_max_clust_count_br_factor = br_factor
                min_max_clust_count_threshold = threshold
                min_max_clust_count_cluster_count = cluster_count
                print(min_max_clust_count,
                      min_max_clust_count_cluster_count,
                      min_max_clust_count_threshold,
                      min_max_clust_count_br_factor)
        print('Thresh done')


### Clustering results plotting

In [ ]:
fig = plt.figure(facecolor='black')
ax = fig.add_subplot(111, frame_on=False)

m.scatter([geom.x for geom in list(station_points)], [geom.y for geom in list(station_points)],
          5, marker='.', lw=.25, facecolor='#33ccff', edgecolor='w', alpha=0.9, antialiased=True, zorder=3)

colors = list(mpl_colors.cnames.keys())
shuffle(colors)
# colors = ['red', 'white', 'blue', 'yellow', 'green', 'cyan', 'purple']

for j in range(len(routes_coord_cur[:400])):
    route_coords = routes_coord_cur[j]
    for i in range(0, len(route_coords) - 1):
#         if idxs[j] == 6:
            point1, point2 = route_coords[i], route_coords[i + 1]
            map_point1, map_point2 = Point(m(point1[0], point1[1])), Point(m(point2[0], point2[1]))
            xs, ys = [map_point1.x, map_point2.x], [map_point1.y, map_point2.y]
            alpha = (i + 1) / len(route_coords)

#             m.scatter(xs, ys, 20, marker='.', lw=.0,
#                       facecolor='red', edgecolor='w', alpha=0.9, antialiased=True, zorder=5)

#             plt.plot(xs, ys, linestyle='-', color=colors[idxs[j] % len(colors)], alpha=alpha + 0.2)
#             plt.plot(xs, ys, linestyle='-', color=colors[j % len(colors)], alpha=0.9)

#             max_sim_route_idx = max_sim_route_idxs[0]
            if j == 171: # max_sim_route_idx[0]:
                plt.plot(xs, ys, linestyle='-', color='red',  alpha=1)
            elif j == 350: # max_sim_route_idx[1]:
                plt.plot(xs, ys, linestyle='-', color='blue', alpha=1)

print('Plotting done')

# ax.add_collection(PatchCollection(df_map['patches'].values, match_original=True))

# fig.set_size_inches(img_width_inches, img_height_inches)
# plt.savefig(
#     "./data_maps_output/routes {}.png".format(int(time.time())),
#     dpi=200, alpha=True, facecolor=fig.get_facecolor())

### Multiple clusters resulting plotting

In [ ]:
for cluster_count in range(21, 22):
#     algo = KMeans(n_clusters=cluster_count, random_state=42)
#     algo = AgglomerativeClustering(n_clusters=cluster_count)
    algo = Birch(threshold=70, branching_factor=70, n_clusters=cluster_count)
    idxs = algo.fit_predict(dist_matrix)

    cur_dir_path = './data_maps_output/routes {}_{}'.format(int(time.time()), cluster_count)
    cur_dir = os.makedirs(cur_dir_path)

    for cluster_idx in range(cluster_count):
        fig = plt.figure(facecolor='black')
        ax = fig.add_subplot(111, frame_on=False)

        m.scatter([geom.x for geom in list(station_points)], [geom.y for geom in list(station_points)],
                  5, marker='.', lw=.25, facecolor='#33ccff', edgecolor='w', alpha=0.9, antialiased=True, zorder=3)

        colors = list(mpl_colors.cnames.keys())
        shuffle(colors)
#         colors = ['red', 'white', 'blue', 'yellow', 'green', 'cyan', 'purple']

        for j in range(len(routes_coord_cur[:400])):
            route_coords = routes_coord_cur[j]
            for i in range(0, len(route_coords) - 1):
                if idxs[j] == cluster_idx:
                    point1, point2 = route_coords[i], route_coords[i + 1]
                    map_point1, map_point2 = Point(m(point1[0], point1[1])), Point(m(point2[0], point2[1]))
                    xs, ys = [map_point1.x, map_point2.x], [map_point1.y, map_point2.y]
                    alpha = (i + 1) / len(route_coords)

#                     plt.plot(xs, ys, linestyle='-', color=colors[idxs[j] % len(colors)], alpha=alpha)
                    plt.plot(xs, ys,linestyle='-', color=colors[j % len(colors)], alpha=1)

        print('Plotting done')

        # ax.add_collection(PatchCollection(df_map['patches'].values, match_original=True))

        fig.set_size_inches(img_width_inches, img_height_inches)
        plt.savefig(cur_dir_path + "/routes {}.png".format(int(time.time())),
                    dpi=200, alpha=True, facecolor=fig.get_facecolor())

### Cluster centroids plotting

In [ ]:
fig = plt.figure(facecolor='black')
ax = fig.add_subplot(111, frame_on=False)

colors = ['red', 'white', 'blue', 'yellow', 'green', 'cyan', 'purple']
# colors = list(mpl_colors.cnames.keys())
# shuffle(colors)

m.scatter([geom.x for geom in list(station_points)], [geom.y for geom in list(station_points)],
          5, marker='.', lw=.25, facecolor='#33ccff', edgecolor='w', alpha=0.9, antialiased=True, zorder=3)

# Work for centroids of the same length only
centroids = np.array([[0.0] * len(routes_coord_cur_flat[0])] * len(algo.cluster_centers_))
routes_per_cluster = [0] * len(algo.cluster_centers_)
for j, route_coords in enumerate(routes_coord_cur_flat):
    centroids[idxs[j]] += route_coords
    routes_per_cluster[idxs[j]] += 1
    
centroids = \
    [centroid / (routes_per_cluster[i] if routes_per_cluster[i] != 0 else -1) 
     for i, centroid in enumerate(centroids)]

for j in range(len(centroids)):
    centroid = centroids[j]
    for i in range(0, len(centroid) - 4, 2):
        point1, point2 = [centroid[i], centroid[i + 1]], [centroid[i + 2], centroid[i + 3]]
        map_point1 = Point(m(point1[0], point1[1]))
        map_point2 = Point(m(point2[0], point2[1]))
        xs = [map_point1.x, map_point2.x]
        ys = [map_point1.y, map_point2.y]
        plt.plot(xs, ys, linestyle='-', color=colors[j % len(colors)])

print('Plotting done')

# ax.add_collection(PatchCollection(df_map['patches'].values, match_original=True))

fig.set_size_inches(img_width_inches, img_height_inches)
plt.savefig("data_maps_output/routes {}.png".format(int(time.time())),
            dpi=200, alpha=True, facecolor=fig.get_facecolor())